In [1]:
import torch
import torch.fx as fx
from transformers import ViTImageProcessor, ViTModel

/home/dboy/open_source/play_torch.fx/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
inputs = processor(images=image, return_tensors="pt")

In [3]:
try:
    gm = fx.symbolic_trace(model)
except fx.proxy.TraceError as err:
    print(err)

symbolically traced variables cannot be used as inputs to control flow


In [4]:
from copy import deepcopy

In [5]:
class GraphCollector:

    def __init__(self):
        self._subgraphs = []

    def reset(self):
        self._subgraphs.clear()

    def __call__(self, gm: fx.GraphModule, sameple_inputs):
        self._subgraphs.append(deepcopy(gm.graph))
        return gm.forward
        
    @property
    def subgraphs(self):
        return self._subgraphs[:]

In [6]:
collector = GraphCollector()
model_ = torch.compile(model, backend=collector).to("cuda:0")

In [7]:
collector.reset()
_ = model_(**{k: v.to("cuda:0") for k, v in inputs.items()})

In [8]:
graph = collector.subgraphs[0]

In [9]:
graph

In [44]:
nodes = [n for n in graph.nodes] # nodes in topological ordering

In [73]:
node = nodes[0]

In [50]:
node.op, node.all_input_nodes, node.users

('placeholder', [], {conv2d: None})

In [74]:
node.name

'l_pixel_values_'

In [75]:
node.target

'L_pixel_values_'

In [52]:
type(node).__module__

'torch.fx.node'

In [65]:
node.meta

{'stack_trace': '  File "/home/dboy/open_source/play_torch.fx/.venv/lib/python3.11/site-packages/transformers/models/vit/modeling_vit.py", line 620, in forward\n    if pixel_values is None:\n',
 'example_value': FakeTensor(..., device='cuda:0', size=(1, 3, 224, 224)),
 'tensor_dict': {},
 'grapharg': GraphArg(source=LocalSource(local_name='pixel_values', cell_or_freevar=False), _example=<torch.utils.weak.TensorWeakRef object at 0x7969d4079b90>, pass_arg_as_tensor=False, fake_tensor=FakeTensor(..., device='cuda:0', size=(1, 3, 224, 224)), is_tensor=True, example_strong_ref=None)}

In [66]:
graph.create_node?

Signature:
graph.create_node(
    op: str,
    target: 'Target',
    args: Optional[Tuple[ForwardRef('Argument'), ...]] = None,
    kwargs: Optional[Dict[str, ForwardRef('Argument')]] = None,
    name: Optional[str] = None,
    type_expr: Optional[Any] = None,
) -> torch.fx.node.Node
Docstring:
Create a ``Node`` and add it to the ``Graph`` at the current insert-point.
Note that the current insert-point can be set via :meth:`Graph.inserting_before`
and :meth:`Graph.inserting_after`.

Args:
    op (str): the opcode for this Node. One of 'call_function', 'call_method', 'get_attr',
        'call_module', 'placeholder', or 'output'. The semantics of these opcodes are
        described in the ``Graph`` docstring.

    args (Optional[Tuple[Argument, ...]]): is a tuple of arguments to this node.

    kwargs (Optional[Dict[str, Argument]]): the kwargs of this Node

    name (Optional[str]): an optional string name for the ``Node``.
        This will influence the name of the value assigned to i

In [69]:
node.target

'L_pixel_values_'

In [71]:
node = graph.create_node("placeholder", target="my_ph_value", name="my_ph")

In [72]:
node.is_impure()

True

In [84]:
set((n.target, n.name) for n in graph.nodes)

{('L_pixel_values_', 'l_pixel_values_'),
 ('L_self_modules_embeddings_modules_patch_embeddings_modules_projection_parameters_bias_',
  'l_self_modules_embeddings_modules_patch_embeddings_modules_projection_parameters_bias_'),
 ('L_self_modules_embeddings_modules_patch_embeddings_modules_projection_parameters_weight_',
  'l_self_modules_embeddings_modules_patch_embeddings_modules_projection_parameters_weight_'),
 ('L_self_modules_embeddings_parameters_cls_token_',
  'l_self_modules_embeddings_parameters_cls_token_'),
 ('L_self_modules_embeddings_parameters_position_embeddings_',
  'l_self_modules_embeddings_parameters_position_embeddings_'),
 ('L_self_modules_encoder_modules_layer_modules_0_modules_attention_modules_attention_modules_key_parameters_bias_',
  'l_self_modules_encoder_modules_layer_modules_0_modules_attention_modules_attention_modules_key_parameters_bias_'),
 ('L_self_modules_encoder_modules_layer_modules_0_modules_attention_modules_attention_modules_key_parameters_weight_

In [85]:
name2node_map = {
    n.name: n
    for n in graph.nodes
}

In [88]:
node = name2node_map["linear_8"]

In [97]:
node.target?

Docstring:
linear(input, weight, bias=None) -> Tensor

Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.

This operation supports 2-D :attr:`weight` with :ref:`sparse layout<sparse-docs>`


.. warning::
    Sparse support is a beta feature and some layout(s)/dtype/device combinations may not be supported,
    or may not have autograd support. If you notice missing functionality please
    open a feature request.

This operator supports :ref:`TensorFloat32<tf32_on_ampere>`.

Shape:

    - Input: :math:`(*, in\_features)` where `*` means any number of
      additional dimensions, including none
    - Weight: :math:`(out\_features, in\_features)` or :math:`(in\_features)`
    - Bias: :math:`(out\_features)` or :math:`()`
    - Output: :math:`(*, out\_features)` or :math:`(*)`, based on the shape of the weight
Type:      builtin_function_or_method

In [96]:
node.args

(layer_norm_2,
 l_self_modules_encoder_modules_layer_modules_1_modules_attention_modules_attention_modules_value_parameters_weight_,
 l_self_modules_encoder_modules_layer_modules_1_modules_attention_modules_attention_modules_value_parameters_bias_)

In [94]:
name2node_map["x_4"]

x_4

In [99]:
ph = name2node_map["l_self_modules_encoder_modules_layer_modules_1_modules_attention_modules_attention_modules_value_parameters_weight_"]

In [103]:
ph.meta["example_value"].shape

torch.Size([768, 768])

In [106]:
node.meta["example_value"].shape

torch.Size([1, 197, 768])